In [1]:
import numpy as np
from scipy.linalg import lu

In [440]:
tol = 1e-9

def lup(matrix):
    assert matrix.shape[0] == matrix.shape[1]
    n = matrix.shape[0]
    L = np.eye(n, dtype=matrix.dtype)
    U = np.copy(matrix)
    P = np.eye(n, dtype=matrix.dtype)
    for i in range(n):
        if np.abs(U[i,i]) < tol:
            for k in range(i + 1, n):
                if np.abs(U[k, i]) > tol:
                    U[[i,k]] = U[[k, i]]
                    L[[i,k]] = L[[k, i]]
                    L[:, [i,k]] = L[:, [k,i]]
                    P[[i,k]] = P[[k, i]]
                    break

        for j in range(i + 1, n):
            coef = U[j,i]/U[i,i]
            U[j, :] -= coef * U[i, :]
            L[j, i] = coef
    return L, U, P

# A = np.array([[1, 4, -3], [-2, 8, 5], [3, 4, 7]])
A = np.array([[3, 9, 6], [4, 12, 12], [1, -1, 1]], dtype=np.float32)

# A = np.random.randint(100, size=[5,5])
# A[3] = A[4]

print(A)
l, u, p = lup(A)
print("lu\n", l, "\n", u, "p\n", p)
np.dot(p.T, np.dot(l, u))

[[  3.   9.   6.]
 [  4.  12.  12.]
 [  1.  -1.   1.]]
lu
 [[ 1.          0.          0.        ]
 [ 0.33333334  1.          0.        ]
 [ 1.33333337 -0.          1.        ]] 
 [[ 3.  9.  6.]
 [ 0. -4. -1.]
 [ 0.  0.  4.]] p
 [[ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]]


array([[  3.,   9.,   6.],
       [  4.,  12.,  12.],
       [  1.,  -1.,   1.]], dtype=float32)

In [460]:
def forwardsub(l, b):
    """
        l -- lower triangular square matrix nxn
        b -- matrix nx1
        
        Returns:
        x -- nx1 vector corresponding to lx=b
    """
    n = l.shape[0]
    x = np.zeros([n], dtype=l.dtype)
    for i in range(n):
        bb = np.dot(x, l[i])
        if np.abs(l[i,i]) < tol:
            raise "Divison by zero"
        x[i] = (b[i,0] - bb)/l[i,i]
    return x.reshape(n, -1)

x = forwardsub(l, np.array([[2,4,5.0]]).T)
np.dot(l, x)

array([[ 2.],
       [ 4.],
       [ 5.]], dtype=float32)

In [471]:
def backwardsub(u, b):
    """
        u -- upper triangular square matrix nxn
        b -- matrix nx1
        
        Returns:
        x -- nx1 vector corresponding to lx=b
    """
    n = l.shape[0]
    x = np.zeros([n], dtype=u.dtype)
    for i in range(n - 1, -1, -1):
        bb = np.dot(x, u[i])
        if np.abs(u[i,i]) < tol:
            raise "Divison by zero"
        x[i] = (b[i,0] - bb)/u[i,i]
    return x.reshape(n, -1)

x = backwardsub(u, np.array([[2,4,5]]).T)
np.dot(u, x)

array([[ 2.],
       [ 4.],
       [ 5.]], dtype=float32)

# 1

Kakve je implementirana u np.allclose ( http://docs.scipy.org/doc/numpy/reference/generated/numpy.allclose.html ) jer dolazi do numericke nestabilnosti buduci da nije moguce zapisivati float brojeve u beskonacnoj preciznosti. Tj. gledamo i/ili relativu i apsolutnu razliku

# 2

In [493]:
A = np.array([[3, 9, 6], [4, 12, 12], [1, -1, 1.0]])
b = np.array([[12,12,1.0]]).T

l, u, p = lup(A)
xp = forwardsub(l, np.dot(p, b))
x = backwardsub(u, xp)

print(np.dot(A, x))
print(np.allclose(np.dot(A, x), b))

[[ 12.]
 [ 12.]
 [  1.]]
True


# 3

In [476]:
B = np.array([[1,2,3],[4,5,6],[7,8,9.0]])
l, u, p = lup(B)
print(p)
print(np.dot(l, u))
print(l, "\n", u) 
# Matrica B je singularna jer det(U) == 0 te det(A) == 0

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]
[[ 1.  0.  0.]
 [ 4.  1.  0.]
 [ 7.  2.  1.]] 
 [[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  0.]]


# 4 

In [494]:
A = np.array([
        [0.000001, 3000000, 2000000], 
        [1000000, 2000000, 3000000], 
        [2000000, 1000000, 2000000]])
b = np.array([[12000000.000001, 14000000, 10000000]]).T


l, u, p = lup(A)
xp = forwardsub(l, np.dot(p, b))
x = backwardsub(u, xp)

print(np.dot(A, x)) # 32bitni float diskrepancija
print(np.allclose(np.dot(A, x), b))

[[ 12000000.000001  ]
 [ 14000560.30184122]
 [ 10000736.7756374 ]]
False


In [495]:
A = np.array([
        [0.000001, 3000000, 2000000], 
        [1000000, 2000000, 3000000], 
        [2000000, 1000000, 2000000]], dtype=np.float128)
b = np.array([[12000000.000001, 14000000, 10000000]], dtype=np.float128).T


l, u, p = lup(A)
xp = forwardsub(l, np.dot(p, b))
x = backwardsub(u, xp)

print(np.dot(A, x)) # 128bitni float diskrepancija.. svejedno
print(np.allclose(np.dot(A, x), b))

[[ 12000000.0]
 [ 14000000.0]
 [ 10000001.0]]
True


# 5

In [496]:
A = np.array([
        [0, 1, 2.0], 
        [2, 0, 3], 
        [3, 5, 1]])
b = np.array([[6, 9, 3]]).T


l, u, p = lup(A)
xp = forwardsub(l, np.dot(p, b))
x = backwardsub(u, xp)

print(x)

print(p) # permutacija je koristena

print(np.dot(A, x)) 

# Rjesenje mi izgleda prilicno ok, vjerojatno ima neke sitne 
# greske an ko zna kojem decimalnom mjestu
print(np.allclose(np.dot(A, x), b))

[[ 0.]
 [ 0.]
 [ 3.]]
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]
[[ 6.]
 [ 9.]
 [ 3.]]
True


# 6

Rješavanje sljedećeg sustava moglo bi zadati problema vašoj implementaciji. O čemu to ovisi? 

Jer imamo jako velike i jako malene brojeve, sto mogu uzrokovati probleme

In [497]:
A = np.array([
        [4000000000, 1000000000, 3000000000], 
        [4, 2, 7], 
        [0.0000000003, 0.0000000005, 0.0000000002]])
b = np.array([[9000000000, 15, 0.0000000015]]).T

# Izbjegavanje velikih raspona brojeva
Trans = np.diag([1/1000000000, 1, 1/0.0000000005])
A = np.dot(Trans, A)
b = np.dot(Trans, b)

l, u, p = lup(A)
xp = forwardsub(l, np.dot(p, b))
x = backwardsub(u, xp)

print(x)

print(p) # permutacija je koristena

print(np.dot(A, x)) 
print(np.allclose(np.dot(A, x), b))

[[ 1.]
 [ 2.]
 [ 1.]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[  9.]
 [ 15.]
 [  3.]]
True
